<a href="https://colab.research.google.com/github/mobinapourmoshir/Functional-Deep-Learning/blob/main/Convolutional_Autoencoder_With_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convolutional Autoencoder**

To enable GPU, you need to change runtime type from Runtime section in the menu section above in colab!

In [1]:
import tensorflow as tf, subprocess, sys
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
!nvidia-smi


TF: 2.19.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Wed Nov 12 19:14:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |             

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled.")
    except Exception as e:
        print("Could not set memory growth:", e)
else:
    print("No GPU detected by TF.")


Memory growth enabled.


In [3]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')  # optional speed boost

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

# Data
(x_train, _), (x_test, _) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test  = x_test.astype('float32') / 255.
x_train_cnn = x_train.reshape(-1, 28, 28, 1)
x_test_cnn  = x_test.reshape(-1, 28, 28, 1)

# Model
inp = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(inp)
x = tf.keras.layers.MaxPooling2D((2,2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same')(x)
enc = tf.keras.layers.MaxPooling2D((2,2), padding='same')(x)

x = tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same')(enc)
x = tf.keras.layers.UpSampling2D((2,2))(x)
x = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2,2))(x)
out = tf.keras.layers.Conv2D(1, (3,3), activation='sigmoid', padding='same', dtype='float32')(x)  # dtype only needed if using mixed precision

model = tf.keras.Model(inp, out)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Training (will use GPU automatically if available)
history = model.fit(
    x_train_cnn, x_train_cnn,
    epochs=10, batch_size=256, shuffle=True,
    validation_data=(x_test_cnn, x_test_cnn), verbose=1
)

# Quick check that TF is really placing on GPU
print("Logical devices:", tf.config.list_logical_devices())


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - loss: 0.4964 - val_loss: 0.2997
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2934 - val_loss: 0.2883
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2850 - val_loss: 0.2843
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2810 - val_loss: 0.2807
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2776 - val_loss: 0.2786
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2757 - val_loss: 0.2770
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.2746 - val_loss: 0.2759
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2731 - val_loss: 0.2748
Epoch 9/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.2723 - val_loss: 0.2739
Epoch 10/10
235/235 ━━━━━━━━━━━━━━━━